## Pydantic y Typing

El tipado (o type hinting) se utiliza en Python para declarar el tipo esperado de una variable, una entrada de función o una salida. Esto no solo mejora la legibilidad del código, sino que es la base para herramientas de validación como Pydantic

### Tipos de Colecciones Comunes

Estos tipos se usan para definir qué tipo de datos contiene una estructura:

- str: Tipo string.
  - nombre: str
- int: Tipo entero.
  - edad: int
- bool: Tipo booleano.
  - registrado: bool
- List: para colecciones ordenadas.
  - documentos: List[str]
- Dict: Para mapeos clave-valor complejos.
  - parametros: dict[str, int]
- Optional: Indica que un valor puede ser de un tipo específico o None.
  - peso: Optional[float]
- Union: Define un valor que puede ser de uno o varios tipos diferentes.
  - id: int | str
- Literal: Permite especificar que una variable solo puede tomar un valor exacto de un conjunto de valores fijos.
  - categoria: Literal["ciencia ficcion", "fantasia"]


###  ¿Qué es Pydantic?

**Pydantic** es una biblioteca de Python que proporciona validación de datos en tiempo de ejecución utilizando las anotaciones de tipo estándar de Python. En términos sencillos, te permite **definir un esquema de datos** (un "molde") y luego asegura que los datos entrantes (como el estado) se ajusten a ese esquema.

Su función principal es: **Validación y Configuración de Datos**.


### Uso Clave en LangGraph

En LangGraph, la clase **`State`** (y otros esquemas como `InputState` y `OutputState`) se definen generalmente utilizando:

- **`TypedDict`** (opción más común por rendimiento).
- **Modelos Pydantic** (`BaseModel` o `dataclass`).

Usar Pydantic garantiza que, si un nodo espera recibir un `documentos_relevantes` como una lista de strings, el sistema generará un error si recibe otra cosa (como un entero), previniendo fallos en el flujo del agente.


### Cómo Usar Pydantic para un Estado Básico

Definir un esquema de estado con Pydantic es tan simple como crear una clase que herede de `BaseModel`.

In [ ]:
from pydantic import BaseModel
from typing import List, Optional

# Definición del esquema del estado de nuestro agente
class AgenteEstado(BaseModel):
    # La pregunta inicial del usuario (str, obligatoria)
    pregunta_usuario: str 
    
    # Lista de resultados de búsqueda (str list, por defecto vacía)
    documentos_encontrados: List[str] = [] 
    
    # La respuesta final generada por el LLM (puede ser None al inicio)
    respuesta_final: Optional[str] = None 
estado_ok = AgenteEstado(pregunta_usuario="¿Qué es LangGraph?") # Esto es válido
estado_error = AgenteEstado(pregunta_usuario=123) # Esto generaría un error de validación

## State, Schema, Reducers

### State

Lo primero que hacemos al definir el grafo es definir el **State** del grafo. Este básicamente es el esquema del grafo, la información que se va a almacenar; además de funciones **reducer**, que especifican cómo se aplicarán los cambios al estado.

El esquema del estado será el esquema de entrada de todos los nodos y aristas del grafo. Además, todos los nodos emitirán actualizaciones al estado, los cuales se aplicarán usando la función reducer especificada.

La manera principal para definir el esquema del grafo es usando un `TypedDict`

Por defecto, el grafo tendrá el mismo esquema de entrada y de salida, pero es posible especificar un esquema de entrada y también uno de salida.

### Schema

Típicamente, todos los nodos del grafo utilizan un mismo esquema, es decir, leen y actualizan el mismo State. Pero, hay casos donde se requiere mayor control y personalización, para ajustar el esquema dependiendo de lo que realice cada nodo. Es posible tener nodos que usan y actualizan estados privados.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

# 1. Estado de Entrada: Lo que el usuario nos da (la primera palabra).
class InputState(TypedDict):
    primera_palabra: str

# 2. Estado de Salida: Lo que el grafo devuelve al final (la oración completa).
class OutputState(TypedDict):
    oracion_completa: str

# 3. Estado General: El contexto central compartido.
class OverallState(TypedDict):
    # La palabra inicial que se heredó del InputState
    primera_palabra: str
    # Una variable que usamos para guardar el Sujeto de la oración
    sujeto_oracion: str
    # La variable que devolveremos al final
    oracion_completa: str

# 4. Estado Privado: Variables temporales para el nodo finalizar oración.
class PrivateState(TypedDict):
    # Canal para el verbo de la oración (temporal/privado)
    verbo_oracion: str

In [ ]:
# Node 1: Lee InputState y escribe OverallState
def nodo_crear_sujeto(state: InputState) -> OverallState:
    """Lee la primera palabra y construye la frase del sujeto."""
    
    # Lectura: Solo puede leer 'primera_palabra' de InputState
    inicio = state["primera_palabra"]
    
    # Lógica: Construye el sujeto
    sujeto = inicio + " El sol"
    print(f"Node 1 (Sujeto): '{sujeto}'")
    
    # Escritura: Retorna un diccionario que actualiza el OverallState
    return {"sujeto_oracion": sujeto}

# Node 2: Lee OverallState y ESCRIBE PrivateState (Crea el canal 'verbo_oracion')
def nodo_agregar_verbo(state: OverallState) -> PrivateState:
    """Lee el sujeto y añade el verbo (escribe a un canal privado)."""
    
    # Lectura: Puede leer 'sujeto_oracion' de OverallState
    sujeto = state["sujeto_oracion"]
    
    # Lógica: Define el verbo basado en el sujeto
    verbo = sujeto + " brilla"
    print(f"Node 2 (Verbo): '{verbo}'")

    # Escritura: Retorna un diccionario que se mapea al PrivateState.
    # ¡Aquí se crea el canal 'verbo_oracion' en el estado del grafo!
    return {"verbo_oracion": verbo} 

# Node 3: Lee PrivateState y escribe OutputState
def nodo_finalizar_oracion(state: PrivateState) -> OutputState:
    """Lee el verbo del estado privado y construye la oración final."""
    
    # Lectura: Solo puede leer 'verbo_oracion' de PrivateState
    verbo_completo = state["verbo_oracion"]
    
    # Lógica: Completa la oración con un predicado.
    oracion = verbo_completo + " en el cielo."
    print(f"Node 3 (Final): '{oracion}'")
    
    # Escritura: Retorna el valor al canal 'oracion_completa' del OutputState.
    return {"oracion_completa": oracion}

In [ ]:
# 1. Inicializar el constructor: Usamos OverallState como el esquema central.
builder = StateGraph(
    OverallState,
    input_schema=InputState,
    output_schema=OutputState
)

# 2. Añadir Nodos
builder.add_node("node_1_sujeto", nodo_crear_sujeto)
builder.add_node("node_2_verbo", nodo_agregar_verbo)
builder.add_node("node_3_final", nodo_finalizar_oracion)

# 3. Definir las Aristas
builder.add_edge(START, "node_1_sujeto")
builder.add_edge("node_1_sujeto", "node_2_verbo")
builder.add_edge("node_2_verbo", "node_3_final")
builder.add_edge("node_3_final", END)

# 4. Compilar y Ejecutar
graph = builder.compile()

from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# La entrada solo necesita la clave del InputState: 'primera_palabra'
entrada_usuario = {"primera_palabra": "Hoy"}
print(f"--- Ejecución con Entrada: {entrada_usuario} ---")

resultado = graph.invoke(entrada_usuario)

print("\nResultado Final del Grafo:")
print(resultado) 

La magia de LangGraph reside en cómo une los diferentes esquemas que defines. Esto es lo que permite que el grafo evolucione el estado progresivamente.

#### 1. ¿Por qué `nodo_crear_sujeto` puede escribir en `sujeto_oracion` (OverallState) si solo recibe `InputState`?

* **El Estado del Grafo es una Unión:** Cuando inicializas el grafo con `builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)`, el **Estado Global del Grafo** se convierte en la **unión de todos estos esquemas**.
* **Acceso Total para Escritura:** Un nodo (como `nodo_crear_sujeto`) solo necesita leer las claves que le pide su *input* (`primera_palabra` del `InputState`), pero tiene permiso para **escribir o actualizar cualquier canal** que exista en el estado global del grafo (`OverallState` o sus filtros).
* **En el ejemplo:** El nodo recibe `{"primera_palabra": "Hoy"}` y lo usa para construir y **escribir** el canal `sujeto_oracion` en el `OverallState`, aunque `sujeto_oracion` no estuviera en su esquema de entrada.

#### 2. ¿Cómo `nodo_agregar_verbo` puede escribir en `verbo_oracion` (PrivateState) si solo se inicializó con `OverallState`?

* **Los Nodos Definen Canales Adicionales:** La inicialización de `StateGraph` define el *punto de partida* y los canales clave. Sin embargo, los nodos pueden **declarar canales adicionales** al retornar un tipo de dato que es un esquema (`TypedDict` o `BaseModel`) definido en tu código.
* **Creación Dinámica de Canales:** En este caso, cuando `nodo_agregar_verbo` retorna un `PrivateState` con la clave `verbo_oracion`, LangGraph **añade** `verbo_oracion` como un nuevo canal al estado global. Este canal es ahora accesible por los nodos posteriores que lo requieran (como `nodo_finalizar_oracion`).
* **En el ejemplo:** El canal `verbo_oracion` no existía al inicio, pero `nodo_agregar_verbo` lo crea al retornar `{"verbo_oracion": "Hoy El sol brilla"}`. El grafo acepta esta actualización y hace que ese canal sea parte de su memoria persistente para el resto de la ejecución. 

### Reducers

Los reducers son funciones que se aplican al momento de actualizar el estado del grafo. Cada parámetro o llave en el estado tiene su propio reducer independiente. Si no se le asigna una explícitamente, por defecto se asume que el modo de actualización será **sobreescribir.**

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
from langgraph.graph import StateGraph, START, END

# Estado Central para nuestro Agente de Investigación
class AgentState(TypedDict):
    # CANAL 1: Contador de pasos (Default Reducer: SOBRESCRIBIR)
    pasos_ejecutados: int 
    
    # CANAL 2: Lista de hallazgos (Reducer Personalizado: CONCATENAR/SUMAR)
    # Annotated le dice a LangGraph: "Esto es una lista de strings, 
    # y usa 'add' (concatenación de listas) para combinar valores nuevos y viejos."
    hallazgos_investigacion: Annotated[list[str], add] 
    # hallazgos_investigacion: list[str]

In [ ]:
# Node 1: Simula el inicio de la investigación.
def nodo_busqueda_inicial(state: AgentState) -> dict:
    """Busca en una fuente y actualiza ambos canales."""
    print("Node 1: Búsqueda inicial realizada.")
    
    # Actualización del Canal 1 (pasos_ejecutados)
    pasos = 1
    
    # Actualización del Canal 2 (hallazgos_investigacion)
    hallazgos = ["Definición de LangGraph (Fuente A)"]
    
    return {"pasos_ejecutados": pasos, "hallazgos_investigacion": hallazgos}

# Node 2: Simula una segunda búsqueda para complementar la información.
def nodo_busqueda_complementaria(state: AgentState) -> dict:
    """Busca en una segunda fuente y actualiza ambos canales."""
    print("Node 2: Búsqueda complementaria realizada.")
    
    # Actualización del Canal 1 (pasos_ejecutados)
    pasos = 2 
    
    # Actualización del Canal 2 (hallazgos_investigacion)
    # Este es el nuevo valor a combinar.
    hallazgos = ["Ejemplos de Reductores (Fuente B)"]
    
    return {"pasos_ejecutados": pasos, "hallazgos_investigacion": hallazgos}

In [ ]:
# 1. Inicializar el constructor con el esquema de estado
builder = StateGraph(AgentState)

# 2. Añadir Nodos
builder.add_node("busqueda_1", nodo_busqueda_inicial)
builder.add_node("busqueda_2", nodo_busqueda_complementaria)

# 3. Definir las Aristas
builder.add_edge(START, "busqueda_1")
builder.add_edge("busqueda_1", "busqueda_2")
builder.add_edge("busqueda_2", END) # Finalizamos para ver el resultado

# 4. Compilar y Ejecutar
graph = builder.compile()

from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# Ejecución
print("--- Ejecución del Flujo de Reductores ---")

resultado_final = graph.invoke({})

print("\nResultado Final del Estado:")
print(resultado_final)

#### El Canal `pasos_ejecutados` (Comportamiento por Defecto: SOBRESCRIBIR)

  * **Definición:** Este canal fue tipado como `int` y **no se le asignó un reductor** explícito con `Annotated`.
  * **Actualización del Node 1:** `pasos_ejecutados` se establece en `1`.
      * *Estado después del Node 1:* `{'pasos_ejecutados': 1, 'hallazgos_investigacion': ['...']}`
  * **Actualización del Node 2:** El nodo retorna `{"pasos_ejecutados": 2}`. Como el modo es **sobrescribir**, el valor `2` reemplaza al valor `1`.
  * **Resultado Final:** El valor de `pasos_ejecutados` es $\mathbf{2}$.

#### El Canal `hallazgos_investigacion` (Comportamiento Personalizado: CONCATENAR)

  * **Definición:** Este canal fue tipado con `Annotated[list[str], add]`, donde `operator.add` para listas es la **concatenación**.
  * **Actualización del Node 1:** El canal se establece en `['Definición...']`.
      * *Estado después del Node 1:* `{'pasos_ejecutados': 1, 'hallazgos_investigacion': ['Definición de LangGraph (Fuente A)']}`
  * **Actualización del Node 2:** El nodo retorna `{"hallazgos_investigacion": ['Ejemplos...']}`. LangGraph no sobrescribe; en su lugar, llama al reductor:
    $$\text{add} (\text{Valor Viejo}, \text{Valor Nuevo})$$
    $$\text{add} (['Definición...'], ['Ejemplos...']) \rightarrow ['Definición...', 'Ejemplos...']$$
  * **Resultado Final:** La lista contiene ambos elementos. Esto asegura que el agente **nunca olvide** los resultados de sus búsquedas previas, manteniendo una memoria acumulativa.

## Nodes

En LangGraph, un nodo no es más que una función de Python (tanto asincrónica como sincrónica) que acepta 3 argumentos:

1. `state`: El estado del Grafo
2. `config`: Un objeto de tipo `RunnableConfig` que contiene información de configuración como el `thread_id` e información de rastreo como  `tags`
3. `runtime`: Un objeto de tipo `Runtime` que contiene información del contexto de la ejecución y también información como `store`y`stream_writer`

Agregamos nodos al grafo usando el método `add_node`

In [ ]:
# Node 1: Simula el inicio de la investigación.
def nodo_busqueda_inicial(state: AgentState) -> dict:
    """Busca en una fuente y actualiza ambos canales."""
    print("Node 1: Búsqueda inicial realizada.")
    
    # Actualización del Canal 1 (pasos_ejecutados)
    pasos = 1
    
    # Actualización del Canal 2 (hallazgos_investigacion)
    hallazgos = ["Definición de LangGraph (Fuente A)"]
    
    return {"pasos_ejecutados": pasos, "hallazgos_investigacion": hallazgos}

# Node 2: Simula una segunda búsqueda para complementar la información.
def nodo_busqueda_complementaria(state: AgentState) -> dict:
    """Busca en una segunda fuente y actualiza ambos canales."""
    print("Node 2: Búsqueda complementaria realizada.")
    
    # Actualización del Canal 1 (pasos_ejecutados)
    pasos = 2 
    
    # Actualización del Canal 2 (hallazgos_investigacion)
    # Este es el nuevo valor a combinar.
    hallazgos = ["Ejemplos de Reductores (Fuente B)"]
    
    return {"pasos_ejecutados": pasos, "hallazgos_investigacion": hallazgos}

builder.add_node("busqueda_1", nodo_busqueda_inicial)
builder.add_node("busqueda_2", nodo_busqueda_complementaria)

### Start Node

El nodo `START` es un nodo especial ya que representa el nodo que envía la entrada del usuario al grafo. Su propósito principal es determinar cuáles nodos deben llamarse primero.

### End Node

El nodo `END` es un nodo especial ya que representa el final del grafo. Este nodo se utiliza para denotar que las aristas ya no tienen más acciones después de que se completan.

## Edges

Las aristas definen la lógica de cómo se enruta el grafo y se decide cuándo parar. Esto es, en gran parte, saber cómo el flujo va a funcionar y cómo los nodos se comunicarán entre ellos.

Existen los siguientes tipo de nodos:

- Normales: Ir directamente de un nodo a otro nodo.
- Condicionales: Llamar a una función para determinar a cuál(es) nodo(s) ir después.
- Punto de entrada: Definir qué nodo llamar primero cuando llega la entrada del usuario.
- Punto de entrada condicional: Llamar a una función para determinar a cuál(es) nodo(s) ir primero cuando llega la entrada del usuario.

Un nodo puede tener múltiples aristas que salen de él. Si esto sucede, todos los nodos destino serán ejecutados en paralelo como parte del siguiente super-paso.

#### Normal Edges

Si **siempre** quieres ir del nodo A al nodo B, entonces se puede usar el método `add_edge` directamente:

In [ ]:
graph.add_edge("node_a", "node_b")

#### Conditional Edges

Si queremos, de forma **opcional**, enrutar a 1 o más nodos, podemos usar el método `add_conditional_edges` . Este método acepta el nombre de un nombre y una función de enrutamiento a llamar luego de que el nodo se haya ejecutado.

In [ ]:
graph.add_conditional_edges("node_a", routing_function)

Similar a los nodos, la función de enrutamiento acepta el estado actual del grafo y retorna un valor.

Por defecto, el valor que retorna la función de enrutamiento es usado como el nombre del nodo (o la lista de nodos) para mandar el estado al siguiente nodo. Todos esos nodos correrán en paralelo como parte del siguiente super-paso.

Opcionalmente, se puede proporcionar un diccionario que mapee la salida de la función de enrutamiento al nombre del siguiente nodo:

In [ ]:
graph.add_conditional_edges("node_a", routing_function, {True: "node_b", False: "node_c"})

#### Entry Point

El entry point es el primer nodo(s) que se ejecuta cuando comienza el grafo. Se puede usar el método `add_edge` desde el nodo `START` hasta el primer nodo a ejecutar al iniciar el grafo.

In [ ]:
from langgraph.graph import START

graph.add_edge(START, "node_a")

#### Conditional Entry Point

Un punto de entrada condicional permite empezar en diferentes nodos dependiendo de lógica personalizada. Se puede usar el método `add_conditional_edges` desde el nodo `START` para lograr esto.

In [ ]:
from langgraph.graph import START

graph.add_conditional_edges(START, routing_function)

Opcionalmente, se puede proporcionar un diccionario que mapee la salida de la función de enrutamiento al nombre del siguiente nodo.

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END

# Estado que usa el grafo
class AgentState(TypedDict):
    pregunta: str
    clasificacion: Literal["SIMPLE", "COMPLEJA"]
    respuesta: str
    
# Tipos de retorno para las funciones de enrutamiento
RoutingDestination = Literal["responder_directo", "investigar"]
StartDestination = Literal["clasificar_simple", "clasificar_compleja"]

In [ ]:
# Node 1: Simula una búsqueda/investigación
def nodo_investigacion(state: AgentState) -> AgentState:
    """Simula una investigación y actualiza la respuesta."""
    print("-> Ejecutando Node: INVESTIGACIÓN (Tarea Compleja)")
    nueva_respuesta = f"INVESTIGADO: La respuesta para '{state['pregunta']}' es detallada."
    return {"respuesta": nueva_respuesta}

# Node 2: Simula una respuesta simple y directa
def nodo_responder_directo(state: AgentState) -> AgentState:
    """Genera una respuesta rápida sin investigación."""
    print("-> Ejecutando Node: RESPUESTA DIRECTA (Tarea Simple)")
    nueva_respuesta = f"DIRECTO: La respuesta para '{state['pregunta']}' es concisa."
    return {"respuesta": nueva_respuesta}

In [ ]:
# Función de Enrutamiento
def enrutar_inicio(state: AgentState) -> StartDestination:
    """Decide el primer nodo a ejecutar basado en la pregunta inicial."""
    pregunta = state['pregunta']
    if len(pregunta) > 20: # Pregunta larga va a clasificación compleja
        return "clasificar_compleja"
    else: # Pregunta corta va a clasificación simple
        return "clasificar_simple"

# Nodos para ser el target de enrutamiento (solo actualizan el estado)
def clasificar_simple(state: AgentState) -> AgentState:
    print("-> Enrutado a CLASIFICACIÓN SIMPLE.")
    return {"clasificacion": "SIMPLE"}

def clasificar_compleja(state: AgentState) -> AgentState:
    print("-> Enrutado a CLASIFICACIÓN COMPLEJA.")
    return {"clasificacion": "COMPLEJA"}

In [ ]:
def enrutar_tarea(state: AgentState) -> RoutingDestination:
    """Decide si la tarea necesita investigación o respuesta directa."""
    clasificacion = state.get('clasificacion')
    
    if clasificacion == "COMPLEJA":
        return "investigar"
    else:
        # Si es SIMPLE (o no tiene una clasificación), responder directamente
        return "responder_directo"

In [ ]:
# 1. Inicializar el constructor
builder = StateGraph(AgentState)

# 2. Añadir Nodos
builder.add_node("clasificar_simple", clasificar_simple)
builder.add_node("clasificar_compleja", clasificar_compleja)
builder.add_node("investigar", nodo_investigacion)
builder.add_node("responder_directo", nodo_responder_directo)

# 3. Definir las Aristas (Edges)

# A. Entry Point Condicional
# La primera ejecución se dirige según 'enrutar_inicio', que mapea a los nodos de clasificación.
builder.add_conditional_edges(
    START, 
    enrutar_inicio, 
    {
        "clasificar_simple": "clasificar_simple", 
        "clasificar_compleja": "clasificar_compleja"
    }
)

# B. Edges Condicionales
# Después de clasificar (simple o compleja), usamos una función para decidir el flujo.
builder.add_conditional_edges(
	"clasificar_simple", 
	enrutar_tarea, 
	{
		"investigar": "investigar", 
		"responder_directo": "responder_directo"
    }
)
builder.add_conditional_edges(
    "clasificar_compleja",
    enrutar_tarea, 
    {
        "investigar": "investigar", 
        "responder_directo": "responder_directo"
	}
)

# C. Edges Normales (Salida)
# Una vez que se responde o investiga, el flujo termina.
builder.add_edge("investigar", END)
builder.add_edge("responder_directo", END)


# 4. Compilar y Ejecutar
graph = builder.compile()

from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# --- Ejecución 1: Pregunta Corta (Flujo Simple) ---
pregunta_corta = "Hola."
print(f"--- Ejecución 1: Pregunta: '{pregunta_corta}' (Flujo SIMPLE) ---")
resultado_corta = graph.invoke({"pregunta": pregunta_corta})
print("\nResultado Final (Corta):", resultado_corta)

In [ ]:
# --- Ejecución 2: Pregunta Larga (Flujo Complejo) ---
pregunta_larga = "Explica en detalle cómo el mecanismo de atención del Transformer impacta en la ventana de contexto."
print(f"--- Ejecución 2: Pregunta: '{pregunta_larga[:40]}...' (Flujo COMPLEJO) ---")
resultado_larga = graph.invoke({"pregunta": pregunta_larga})
print("\nResultado Final (Larga):", resultado_larga)